<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Federated_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install syft==0.2.9 torch torchvision

In [ ]:
import syft as sy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Create a VirtualMachine and Client
alice_vm = sy.VirtualMachine(name="alice")
alice = alice_vm.get_root_client()
bob_vm = sy.VirtualMachine(name="bob")
bob = bob_vm.get_root_client()

# Create a simple dataset and split between two workers
data = torch.tensor([[1, 2], [3, 4], [5, 6], [7, 8]], dtype=torch.float32)
target = torch.tensor([[1], [0], [1], [0]], dtype=torch.float32)
dataset = TensorDataset(data, target)
alice_data, bob_data = torch.utils.data.random_split(dataset, [2, 2])

# Send data to VirtualWorkers
alice_data_ptr = [(d.send(alice), t.send(alice)) for d, t in alice_data]
bob_data_ptr = [(d.send(bob), t.send(bob)) for d, t in bob_data]

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc = nn.Linear(2, 1)

    def forward(self, x):
        return torch.sigmoid(self.fc(x))

model = Net()
optimizer = optim.SGD(model.parameters(), lr=0.01)

def train(data_ptrs, model, optimizer, client):
    model.send(client)
    for data, target in data_ptrs:
        optimizer.zero_grad()
        output = model(data)
        loss = ((output - target) ** 2).mean()
        loss.backward()
        optimizer.step()
    model.get()  # Get model back from the client
    print(f'Loss: {loss.get().item()}')

for epoch in range(5):
    train(alice_data_ptr, model, optimizer, alice)
    train(bob_data_ptr, model, optimizer, bob)